# Initial kernel based on different RNN layers

* **Text preprocessing**

Following things to be tried on the baseline:
    * Add Early Stopping callback
    * Increase max epochs - let EarlyStop do the work
    * Add Tensorboard callback, monitor training
    * Replace LSTM by GRU units and check if it changes anything
    * Add another layer of LSTM/GRU, see if things improve
    * Play around with Dense layers (add/# units/etc.)
    * Find preprocessing rules you could add to improve the quality of the data
    * Use different embeddings

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Permute, GRU, Conv1D, LSTM, Embedding, Dropout, Activation, CuDNNLSTM, CuDNNGRU, concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D, Dot
from keras.optimizers import Adam
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras_tqdm import TQDMNotebookCallback
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from functools import reduce
from keras.layers import Layer, PReLU
from keras import initializers
from sklearn.model_selection import cross_val_predict

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = '../input/'
utility_path = '../utility/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE=f'../utility/crawl-300d-2M.vec'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

In [3]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [5]:
embed_size = 300 # how big is each word vector
max_features = 200000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 200 # max number of words in a comment to use["comment_text"]

In [6]:
train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)


In [7]:
train["comment_text"] = train["comment_text"].str.replace("[.?!]{1,}\s", " <eos> ").str.replace("(\n){1,}", " <eop> ").str.replace("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","")
test["comment_text"] = test["comment_text"].str.replace("[.?!]{1,}\s", " <eos> ").str.replace("(\n){1,}", " <eop> ").replace("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","")

train["comment_text"] = train["comment_text"].str.replace("\n", "")
test["comment_text"] = test["comment_text"].str.replace("\n", "")

#train["comment_text"] = train["comment_text"].str.replace("talk", "").str.replace("==", "")
#test["comment_text"] = test["comment_text"].str.replace("talk", "").str.replace("==", "")

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

In [8]:
train.comment_text.sample(1).values

array(["Well <eos>  <eop> You could for instance beef up 'this with '''that <eos> Cheers,"],
      dtype=object)

In [9]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen, padding='post', truncating='post')
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen, padding='post', truncating='post')

In [10]:
len(embeddings_index)

NameError: name 'embeddings_index' is not defined

In [11]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector


In [12]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.0055286596, 0.34703913)

In [13]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        # self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))),
                        (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        # print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        # return input_shape[0], input_shape[-1]
        return input_shape[0], self.features_dim

In [14]:
class ZeroMaskedEntries(Layer):
    """
    This layer is called after an Embedding layer.
    It zeros out all of the masked-out embeddings.
    It also swallows the mask without passing it on.
    You can change this to default pass-on behavior as follows:

    def compute_mask(self, x, mask=None):
        if not self.mask_zero:
            return None
        else:
            return K.not_equal(x, 0)
    """

    def __init__(self, **kwargs):
        self.support_mask = True
        super(ZeroMaskedEntries, self).__init__(**kwargs)

    def build(self, input_shape):
        self.output_dim = input_shape[1]
        self.repeat_dim = input_shape[2]

    def call(self, x, mask=None):
        #print(mask.shape)
        mask = K.cast(mask, 'float32')
        mask = K.repeat(mask, self.repeat_dim)
        #print(mask.shape)
        mask = K.permute_dimensions(mask, (0, 2, 1))
        return x * mask

    def compute_mask(self, input_shape, input_mask=None):
        return None
    
def mask_aware_mean(x):
    # recreate the masks - all zero rows have been masked
    #mask = K.not_equal(K.sum(K.abs(x), axis=2, keepdims=True), 0)

    # number of that rows are not all zeros
    #n = K.sum(K.cast(mask, 'float32'), axis=1, keepdims=False)
    # compute mask-aware mean of x
    x_mean = K.sum(x, axis=1, keepdims=False)
    #print(x_mean.shape)
    return x_mean

def mask_aware_mean_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 3 
    return (shape[0], shape[2])

In [23]:
def get_model():
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size,
                  weights=[embedding_matrix],
                 trainable=True, mask_zero=True)(inp)
    #x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
    #x = Conv1D(filters=64, kernel_size=2)(x)
    
    #avg_emd = GlobalAveragePooling1D()(x)
    #x = SpatialDropout1D(0.5)(x)
    x = ZeroMaskedEntries()(x)
    x = SpatialDropout1D(0.1)(x)
    #tmp = Bidirectional(CuDNNGRU(128, return_sequences=True, return_state=True))(x)
    #x = tmp[0]
    #state1 = tmp[1]
    #state2 = tmp[2]
    print(x)
    x, state1 = CuDNNGRU(150, return_sequences=True, return_state=True)(x)
    #x, state2 = CuDNNGRU(150, return_sequences=True, return_state=True)(x)
    #x_flat = Flatten()(x)
    #x_dot = GlobalAveragePooling1D()(x_dot)
    #attn_pool = Permute((2,1))(x)
    #attn_pool = Dense(maxlen, activation='tanh')(attn_pool)
    #attn_pool = Dense(maxlen, activation=Activation(K.exp))(attn_pool)
    #attn_pool = Permute((2,1))(attn_pool)
    #attn_pool = GlobalAveragePooling1D()(attn_pool)
    attn_pool = Attention(maxlen)(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPool1D()(x)
    

    #x_dot = Dot(1)([avg_pool, max_pool])
    x = concatenate([ max_pool, attn_pool, state1
                    ])
    #x = avg_pool
    #x = BatchNormalization()(x)
    
    x = Dense(600)(x)
    x = PReLU()(x)
    #x = Dropout(0.1)(x)
    #x = Dense(256)(x)
    #x = PReLU()(x)
    #x = Dropout(0.1)(x)
    #x = Dense(6, activation='softmax')(x)
    x = Dense(6, activation="sigmoid")(x)
    opt = Adam(lr=0.001, decay=0, clipnorm=10)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [21]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['target_str'] = reduce(lambda x,y: x+y, [train[col].astype(str) for col in label_cols])
train['target_str'] = train['target_str'].replace('110101', '000000').replace('110110','000000')
cvlist = list(StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=786).split(train, train['target_str'].astype('category')))

In [ ]:
def lr_decay(epoch):
    if epoch == 0:
        return 0.0015
    if epoch == 1:
        return 0.0008
    if epoch == 2:
        return 0.0008
    if epoch == 3:
        return 0.00001
    
y_preds = np.zeros((len(X_t), len(label_cols)))
LRDecay = LearningRateScheduler(lr_decay)
for tr_index, val_index in cvlist:
    X_train, y_train = X_t[tr_index, :], y[tr_index]
    X_val, y_val = X_t[val_index, :], y[val_index]
    RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)
    model = get_model()
    model.fit(X_train, y_train, batch_size=128, epochs=2, validation_split=0.0, verbose=1, 
              callbacks=[RocAuc, LRDecay])
    y_preds[val_index, :] = model.predict(X_val, batch_size=2048)
    K.clear_session()
    #break

Tensor("spatial_dropout1d_1/cond/Merge:0", shape=(?, 200, 300), dtype=float32)
Epoch 1/2
151552/151592 [============================>.] - ETA: 0s - loss: 0.0521 - acc: 0.9809
 ROC-AUC - epoch: 1 - score: 0.988923 

151592/151592 [==============================] - 68s 447us/step - loss: 0.0521 - acc: 0.9809
Epoch 2/2
145536/151592 [===========================>..] - ETA: 2s - loss: 0.0333 - acc: 0.9864

In [29]:
#roc_auc_score(y, y_preds)
import gc
gc.collect()
K.clear_session()

In [ ]:
test_preds = []
for i in range(4):
    model = get_model()
    model.fit(X_t, y, batch_size=128, epochs=2, validation_split=0.001, verbose=1, 
                  callbacks=[RocAuc, LRDecay])
    y_test_preds1 = model.predict([X_te], batch_size=1024, verbose=1)
    test_preds.append(y_test_preds1)


In [ ]:
test_preds_mean = (test_preds[0] * test_preds[1] * test_preds[2] * test_preds[3])**(1/4)

In [ ]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[label_cols] = test_preds_mean
sample_submission.to_csv('nn_submission_v2.csv', index=False)

In [25]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
y_trues = train[label_cols].values
y_preds2 = np.zeros((X_t.shape[0], len(label_cols)))
y_test_preds2 = np.zeros((X_te.shape[0], len(label_cols)))
for i, col in enumerate(label_cols):
    y = y_trues[:, i]
    #model = RandomForestClassifier(n_estimators=100, max_depth=6, min_samples_leaf=50, class_weight='balanced', n_jobs=-1)
    model = lgb.LGBMClassifier(n_estimators=150, num_leaves=8, learning_rate=0.05, 
                               subsample=0.7, colsample_bytree=0.9)
    y_preds2[:, i] = cross_val_predict(model, y_preds, y, cv=cvlist, n_jobs=1, method='predict_proba')[:,1]
    y_test_preds2[:, i] = model.fit(y_preds, y).predict_proba(y_test_preds)[:,1]
    print("Score for class {} is {}".format(col, roc_auc_score(y, y_preds2[:, i])))
print("Over auc score", roc_auc_score(y_trues, y_preds2))

Score for class toxic is 0.9791088111561013
Score for class severe_toxic is 0.9896361881404786
Score for class obscene is 0.989792727484462
Score for class threat is 0.9834375512104746
Score for class insult is 0.9851961777705942
Score for class identity_hate is 0.9837426177272286
Over auc score 0.9851523455815565


In [44]:
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[label_cols] = test_preds_mean
sample_submission.to_csv('nn_submission.csv', index=False)

In [45]:
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998639,6.222732e-01,0.986446,5.110442e-02,9.549904e-01,5.519497e-01
1,0000247867823ef7,0.000003,3.789467e-08,0.000001,1.027940e-09,2.875290e-07,1.663085e-07
2,00013b17ad220c46,0.000370,3.100435e-06,0.000017,3.314079e-05,1.175709e-05,2.276327e-05
3,00017563c3f7919a,0.000001,2.953290e-07,0.000007,3.394860e-09,1.432248e-06,4.007862e-07
4,00017695ad8997eb,0.000042,7.988882e-07,0.000020,5.988817e-07,3.980357e-06,2.081103e-06


In [ ]:
sample_submission[label_cols] = y_test_preds2
sample_submission.to_csv('nn_lgbmeta_submission.csv', index=False)